##### 实用技巧：

1. 布尔值也是**数字**：`True`等价于`1`，`False`等价于`0`。

```python
#可配合sum函数在需要计算总数时简化操作
l = [1,2,4,5,7]
total_ou = sum(i % 2 == 0 for i in l)

#将某个布尔值当做列表的下标使用，可实现类似三元表达式的目的
result = ["Python", "Javascript"][2 > 1]

```

2. **使用括号将长字符串包起来，可以随意拆行**。

```python
s = (
    "There is something really bad happened during the process. "
    "Please contact your administrator."
)

def main():
    logger.info(
        "There is something really bad happened during the process. "
        "Please contact your administrator."
    )
```
3. 使用`textwrap`库解决带缩进层级的代码里的长字符串问题。

```python
for textwrap import dedent

def main():
    if user.is_active:
        # dedent 将会缩进整段文字最左边的空字符串
        message = dedent("""\
            Welcome, today's movie list:
            - Jaw (1975)
            - The Shining (1980)
            - Saw (2004)""")
```
4. 特别大的数字，可以通过在中间添加下划线提高可读性。

```python
# 以“千”为单位划分数字
bill = 10_000_000.0
# 16进制数字同样有效，4个一组更易读
num1 = 0xCAFE_F00D
# 二进制也有效
num2 = 0b_0011_1111_0100_1110
# 处理字符串的时候也会正确处理下划线
num3 = int('0b_1111_0000', 2)
```

5. 使用“无穷大”float("inf")：`float("-inf") < 任意数值 < float("inf")`

```python
# A.根据年龄升序排序，没有提供年龄的放在最后边
users = {"tom": 19, "jenny": 13, "jack": None, "andrew": 43}
sorted_users = sorted(users.keys(), key=lambda user: users.get(user) or float('inf'))

# B. 作为循环初始值，简化第一次判断逻辑(特别是在不能明确实际数据的范围时)
max_num = float('-inf')
# 找到列表中最大的数字
for i in [23,71,3,21,8]:
    if i > max_num:
        max_num = i
```

##### 常见误区

1. `value += 1`并非线程安全：

“线程安全”：通常被用来形容**某个行为或者某类数据结构，可以在多线程环境下被共享使用并产生预期内的结果**。

一个典型的满足“线程安全”的模块是`queue`队列模块。

> 多线程环境下的编码非常复杂，要足够谨慎，不能相信自己的直觉。

### 容器之道

**容器**：*对专门用来装其他对象的数据类型的统称。*

容器的两个层面：
* **底层实现**：内置容器类型使用了什么数据结构？某项操作如何工作？
* **高层抽象**：什么决定了某个对象是不是容器？哪些行为定义了容器？

常见的内建容器类型：
* 列表（list）
* 元组（tuple）
* 字典（dict）
* 集合（set）

#### 底层看容器

##### 写更快的代码

1. 避免频繁扩充列表/创建新列表

> 在Python的实现细节里面，列表的内存是按需分配的，当某个列表当前拥有的内存不够时，便会触发内存扩容逻辑。而分配内存是一项昂贵的操作。

避免过于频繁的内存分配：
* 更多的使用`yield`关键字，返回生成器对象
* 尽量使用生成器表达式替代列表推导表达式
    * 生成器表达式：`(i for i in range(100))`
    * 列表推导表达式：`[i for i in range(100)]`
* 尽量使用模块提供的懒惰对象：
    * 使用`refinditer`替代`re.findall`
    * 直接使用可迭代的文件对象：`for line in fp`，而不是`for line in fp.readlines()`

2. 在列表头部操作多的场景使用`deque`模块

如需在列表头部操作多，使用[`collections.deque`](https://docs.python.org/3.7/library/collections.html#collections.deque)类型替代列表。

3. 使用集合/字典来判断成员是否存在

> 判断成员是否存在于某个容器时，用集合比列表更合适，因为`item in {...}`的时间复杂度是`O(1)`，因为字典与集合都是基于哈希表（Hash Table）数据结构实现的。


> Hint：关于容器类型的而时间复杂度相关内容：[TimeComplexity](https://wiki.python.org/moin/TimeComplexity)

In [1]:
# 利用生成器表达式替代列表推导表达式
test = (i for i in range(1000000000000000000000))

#### 高层看容器

每个内置容器类型，其实就是满足了多个接口定义的组合实体。

面向对象编程中最重要的原则之一：**面向接口而非具体实现来编程**。

##### 写扩展性更好的代码

面向容器接口编程

**让函数依赖“可迭代对象”这个抽象概念，而非实体列表类型。**



In [4]:
import typing
# 使用生成器特性，实现将超过一定长度的评论用省略号替代
def add_ellipsis_gen(comments: typing.Iterable[str], max_length: int = 12):
    """如果可迭代评论里的内容超过max_length，剩下的字符用省略号替代
    """
    for comment in comments:
        comment = comment.strip()
        if len(comment) > max_length:
            yield comment[:max_length] + '...'
        else:
            yield comment


In [7]:
comments = [
    "Implementation note",
    "Changed",
    "ABC for generator",
]
print("\n".join(add_ellipsis_gen(comments)))

Implementati...
Changed
ABC for gene...


In [6]:
comments = (
    "Implementation note",
    "Changed",
    "ABC for generator",
)
print("\n".join(add_ellipsis_gen(comments)))

Implementati...
Changed
ABC for gene...


无论评论是来自列表、元组或者某个文件，新函数都可以轻松满足。

In [8]:
# 处理放在元组里的评论
comments = ("Implementation note", "Changed", "ABC for generator")
print("\n".join(add_ellipsis_gen(comments)))


Implementati...
Changed
ABC for gene...


In [9]:
# 处理放在文件里的评论
with open("comments.txt") as fp:
    for comment in add_ellipsis_gen(fp):
        print(comment)

Implementati...
Changed
ABC for gene...


将依赖由某个具体的容器类型改为抽象接口后，函数的适用面变得更广了。

**从高层来看，什么定义了容器？**
> **各个容器类型实现的接口协议定义了容器**。不同的容器类型在我们眼里，应该是*是否可以迭代、是否可以修改、有没有长度*等各种特性的组合。

> **更多关注容器的抽象属性，而非容器类型本身**。



#### 常用技巧

1. 使用元组改善分支代码

有时，代码里会出现超过三个分支的`if/else`：

In [18]:
import time

def from_now(ts):
    """接收一个过去的时间戳，返回距离当前时间的相对时间文字描述
    """
    now = time.time()
    seconds_delta = int(now - ts)
    if seconds_delta < 1:
        return "less than 1 second ago"
    elif seconds_delta < 60:
        return "{} seconds ago".format(seconds_delta)
    elif seconds_delta < 3600:
        return "{} minutes ago".format(seconds_delta // 60)
    elif seconds_delta < 3600 * 24:
        return "{} hours ago".format(seconds_delta // 3600)
    else:
        return "{} days ago".format(seconds_delta // (3600 * 24))


now = time.time()
print(from_now(now))
print(from_now(now - 24))
print(from_now(now - 600))
print(from_now(now - 7500))
print(from_now(now - 87500))

        

less than 1 second ago
24 seconds ago
10 minutes ago
2 hours ago
1 days ago


在上面的分支代码部分中可以找到一些明显的“**边界**”，**从边界提炼规律是优化这段代码的关键**。

可以将所有的这些边界放在一个有序元组中，然后配合二分查找模板[`bisect`](https://docs.python.org/zh-cn/3.7/library/bisect.html)，可以大大简化整个函数的控制流：

In [19]:
import bisect

# BREAKPOINTS 必须是已经排好序的，不然无法进行二分查找
BREAKPOINTS = (1, 60, 3600, 3600 * 24)
TMPLS = (
    # unit, template
    (1, "less than 1 second ago"),
    (1, "{units} seconds ago"),
    (60, "{units} minutes ago"),
    (3600, "{units} hours ago"),
    (3600 * 24, "{units} days ago"),
)

def from_now(ts):
    """接收一个过去的时间戳，返回距离当前时间的相对时间文字描述
    """
    seconds_delta = int(time.time() - ts)
    unit, tmpl = TMPLS[bisect.bisect(BREAKPOINTS, seconds_delta)]
    return tmpl.format(units=seconds_delta // unit)


now = time.time()
print(from_now(now))
print(from_now(now - 24))
print(from_now(now - 600))
print(from_now(now - 7500))
print(from_now(now - 87500))

        

less than 1 second ago
24 seconds ago
10 minutes ago
2 hours ago
1 days ago


除了用元组可以优化过多的`if/else`分支外，有些情况下字典也能被用来做同样的事情。

2. 在更多地方使用动态解包

动态解包操作是指使用`*`或`**`运算符将可迭代对象“解开”的行为，可以直接用`**`运算符来快速完成字典的合并操作：

In [20]:
user = {**{"name": "piglei"}, **{"movies": ["Fight Club"]}}

print(user)

{'name': 'piglei', 'movies': ['Fight Club']}


可以在普通赋值语句中使用`*`运算符来动态地解包可迭代对象。

3. 使用`next()`函数

`next()`是一个非常实用的内建函数，接收一个迭代器作为参数，然后返回该迭代器的下一个元素。使用它配合生成器表达式，可以高效的实现“*从列表中查找第一个满足条件的成员*”之类的需求。

In [21]:
numbers = [3, 7, 8, 2, 21]
# 获取并 **立即返回** 列表里的第一个偶数
print(next(i for i in numbers if i % 2 ==0))


8


4. 使用有序字典来去重

字典和集合的结构特点保证了它们的成员不会重复，所以它们经常被用来去重。但是去重后会丢失原有列表的顺序（由底层数据结构“哈希表（Hash Table）”的特点决定的。

In [24]:
l = [10, 2, 3, 21, 10, 3]
# 去重但是丢失了顺序
set(l)

{2, 3, 10, 21}

可以使用`collections.OrderedDict`模块去重并保留顺序：

In [25]:
from collections import OrderedDict
list(OrderedDict.fromkeys(l).keys())

[10, 2, 3, 21]

#### 常见误区

1. 当心那些已经枯竭的迭代器

**生成器最大的缺点之一就是：它会枯竭**。当完整遍历过它们后，之后的重复遍历就不能拿到任何新内容了。

In [26]:
numbers = [1, 2, 3]
numbers = (i * 2 for i in numbers)

# 第一次循环会输出2， 4,6
for number in numbers:
    print(number)

# 这次循环什么都不会输出，因为迭代器已经枯竭了
for number in numbers:
    print(number)

2
4
6


Python3里的map、filter内建函数也都有一样的特点。

2. 别在循环体内修改被迭代对象



In [27]:
def remove_even(numbers):
    """去掉列表里所有的偶数
    """
    for i, number in enumerate(numbers):
        if number % 2 == 0:
            # 有问题的代码
            del numbers[i]

numbers = [1, 2, 7, 4, 8, 11]
remove_even(numbers)
print(numbers)

[1, 7, 8, 11]


结果中会多出来一个“8”，当在遍历一个列表的同时修改它，被迭代的对象`numbers`在循环过程中被修改了。

**遍历的下标在不断增长，而列表本身的长度同时又在不断缩减。会导致列表里的一些成员其实根本没有被遍历到**。



### 让函数返回结果的技巧

#### 编程建议

1. 单个函数不要返回多种类型

好的函数一定是“**单一职责（Single responsibility）**”的。

**单一职责意味着一个函数只做好一件事，目的明确**。这样的函数也更不容易在未来因为需求变更而被修改。

**好的函数应该总是提供稳定的返回值，把调用方的处理成本降到最低**。

2. 使用`partial`构造新函数

假设在代码中有一个参数很多的函数`A`，适用性很强，而另一个函数`B`则是完全通过调用`A`来完成工作，是一种类似快捷方式的存在。

针对这样的场景，可以使用`functools`模块里的[`partial()`](https://docs.python.org/3.6/library/functools.html#functools.partial)函数来简化。

`partial(func, *args, **kwargs)`基于传入的函数与可变（位置/关键字）参数来构造一个新函数。**所有对新函数的调用，都会在合并了当前调用参数与构造参数后，代理给原始函数处理**。

3. 抛出异常，而不是返回结果与错误



In [28]:
class CreateItemError(Exception):
    """创建 Item 失败时抛出的异常"""

def create_item(name):
    """创建一个新的 Item
    
    :raises: 当无法创建时抛出 CreateItemError
    """
    if len(name) > MAX_LENGTH_OF_NAME:
        raise CreateItemError('name of item is too long')
    if len(CURRENT_ITEMS) > MAX_ITEMS_QUOTA:
        raise CreateItemError('items if full')
    return Item(name=name)

def create_for_input():
    name = input()
    try:
        item = create_item(name)
    except CreateItemError as e:
        print(f'create item failed: {e}')
    else:
        print(f'item<{name}> created')


4. 谨慎使用`None`返回值

`None`值通常用来表示“**某个应该存在但是缺失的东西**”。

当使用`None`作为函数返回值时，通常是下面3种情况：

* 作为操作类函数的默认返回值：当某个操作类函数不需要任何返回值时，通常就会返回`None`。`None`也是不带任何`return`语句函数的默认返回值。
* 作为某些“意料之中”的可能没有的值：有一些函数的目的通常是去尝试性的做某件事情。视情况不同，最终可能有结果，也可能没有结果。**而对调用方来说，“没有结果”完全是意料之中的事情**。
* 作为调用失败时代表“错误结果”的值：有时，`None`也会经常被我妈用来作为函数调用失败时的默认返回值，比如：

In [35]:
class User():
    def __init__(self, username) -> None:
        self.username = username



def validate_username(username):
    return username

def create_user_from_name(username):
    """通过用户名创建一个 User 实例"""
    if validate_username(username):
        return username
    else:
        return None

user = create_user_from_name("username")
if user:
    print(user)

username


> **函数签名（名称与参数）与None返回值之间是否存在一种“意料之中”的暗示。**

与`None`返回值相比，抛出异常除了拥有上个场景中提到的那些特点外，还有一个额外的优势：**可以在异常信息里提供出现意料之外结果的原因**，是只返回一个`None`值做不到的。

5. 合理使用“空对象模式”

使用`None`值或异常来返回错误结果有一个共同的缺点，就是所有需要使用函数返回值的地方，都必须加上一个`if`或`try/except`防御语句，来判断结果是否正常。



In [36]:
import decimal

class CreateAccountError(Exception):
    """Unable to create a account error"""

class Account:
    """一个虚拟的银行账号"""

    def __init__(self, username, balance):
        self.username = username
        self.balance = balance
    
    @classmethod
    def from_string(cls, s):
        """从字符串初始化一个账号"""
        try:
            username, balance = s.split()
            balance = decimal.Decimal(float(balance))
        except ValueError:
            raise CreateAccountError('input must follow pattern "{ACCOUNT_NAME} {BALANCE}"')
        
        if balance < 0:
            raise CreateAccountError('balance can not be negative')
        return cls(username=username, balance=balance)

def caculate_total_balance(accounts_data):
    """计算所有账号的总金额"""
    result = 0
    for account_string in accounts_data:
        try:
            user = Account.from_string(account_string)
        except CreateAccountError:
            pass
        else:
            result += user.balance
    return result

accounts_data = [
    'piglei 96.5',
    'cotton 21',
    'invalid_data',
    'roland $invalid_balance',
    'alfred -3',
    'soufal 100'
]

print(caculate_total_balance(accounts_data))

        

217.5


在上面的例子中，如果每次调用`Account.from_string`时，都必须使用`try/except`来捕获可能发生的异常。如果项目里需要调用很多次该函数，这部分的工作就会变得非常繁琐。可以使用**空对象模式（Null object pattern）**来改善。

**使用一个符合正常结果接口的“空类型”来替代空值返回/抛出异常，以此来降低调用方出来结果的成本**。



In [37]:
class Account:
    """一个虚拟的银行账号"""

    def __init__(self, username, balance):
        self.username = username
        self.balance = balance
    
    @classmethod
    def from_string(cls, s):
        """从字符串初始化一个账号
        
        :returns: 如果输入合法，返回 Account object， 否则返回 NullAccount
        """
        try:
            username, balance = s.split()
            balance = decimal.Decimal(float(balance))
        except ValueError:
            return NullAccount()

        if balance < 0:
            return NullAccount()
        return cls(username=username, balance=balance)

class NullAccount:
    username = ''
    balance = 0

    @classmethod
    def from_string(cls, s):
        raise NotImplementedError

def caculcate_total_balance(accounts_data):
    """计算所有账号的总金额"""
    return sum(Account.from_string(s).balance for s in accounts_data)

accounts_data = [
    'piglei 96.5',
    'cotton 21',
    'invalid_data',
    'roland $invalid_balance',
    'alfred -3',
    'soufal 100'
]

print(caculate_total_balance(accounts_data))

217.5


重新定义`NullAccount`新类型，用来作为`from_string`失败时的错误结果返回。

调整之后，调用方不必再显式使用`try`语句来处理错误，而是可以假设`Account.from_string`函数总是会返回一个合法的`Account`对象，从而大大简化整个计算逻辑。

6. 使用生成器函数代替返回列表

在函数里返回列表特别常见，通常会先初始化一个列表`results=[]`，然后再循环体内使用`results.append(item)`函数填充它，最后在函数的末尾返回。

可以用生成器函数来简化，就是用`yield item`替代`append`语句。使用生成器的函数通常更简洁、也更具通用性。


In [38]:
def foo_func(items):
    for item in items:
        # ... 处理item后直接使用yield返回
        yield item

7. 限制递归的使用

当函数返回自身调用时，也就是**递归**发生时。但是Python语言对递归支持的非常有限。
首先，Python语言不支持“尾递归优化”，另外Python对最大递归层级数也有严格的限制。

**尽量少些递归**。如果一定要使用递归：
* 函数输入数据规模是否稳定，是否一定不会超过`sys.getrecursionlimit()`规定的最大层数限制
* 是否可以通过使用类似`functools.lru_cache`的缓存工具函数来降低递归层数。